# 热点区域的定义

取车热点区域：在一个时间窗内，取车点在空间聚集的区域。

还车热点区域：在一个时间窗内，还车点在空间聚集的区域。

停放单车热点区域：在一个时间点，处于停放状态的单车在空间聚集的区域。

In [1]:
import pandas as pd
import geopandas
import transbigdata as tbd

In [2]:
move_data = pd.read_csv(r'E:\data\1 上海共享单车2019\move_data.csv')
stop_data = pd.read_csv(r'E:\data\1 上海共享单车2019\stop_data.csv')

In [3]:
move_data.head()

,BIKE_ID,stime,slon,slat,etime,elon,elat
0,0,2019-05-06 06:41:33,121.470761,31.260999,2019-05-06 06:46:24,121.464820,31.255403
1,0,2019-05-06 07:16:55,121.464294,31.254926,2019-05-06 07:24:10,121.455562,31.255618
2,0,2019-05-06 07:48:50,121.455567,31.255606,2019-05-06 08:05:15,121.431458,31.244208
3,0,2019-05-06 09:11:51,121.431436,31.244420,2019-05-06 09:25:27,121.452108,31.243948
4,0,2019-05-06 13:21:27,121.451857,31.243207,2019-05-06 13:31:44,121.431198,31.244091


In [4]:
stop_data.head()

,BIKE_ID,stime,slon,slat,etime,elon,elat
0,0,2019-05-06 00:00:00,121.470761,31.260999,2019-05-06 06:41:33,121.470761,31.260999
1,0,2019-05-07 20:57:54,121.401901,31.242309,2019-05-12 23:59:59,121.401901,31.242309
2,0,2019-05-06 06:46:24,121.464820,31.255403,2019-05-06 07:16:55,121.464294,31.254926
3,0,2019-05-06 07:24:10,121.455562,31.255618,2019-05-06 07:48:50,121.455567,31.255606
4,0,2019-05-06 08:05:15,121.431458,31.244208,2019-05-06 09:11:51,121.431436,31.244420


# 热点区域的订单数据提取

## 取车热点区域

In [5]:
def pickupDataProcess(move_data, startT, endT):
    """输入一周的单车移动数据，指定的日期与指定的时刻"""
    """输出该日期该时间切片内取车pickup位置"""

    data = move_data[(move_data['stime'] < endT) &
                     (move_data['stime'] >= startT)]
    data = data[['slon', 'slat']]

    data['geometry'] = geopandas.points_from_xy(data['slon'], data['slat'])

    data = geopandas.GeoDataFrame(data)
    data = data.set_crs('EPSG:4326')
    data = data.to_crs('EPSG:2416')

    data['x'] = data.geometry.x
    data['y'] = data.geometry.y

    data = data.to_crs('EPSG:4326')

    return data

## 还车热点区域 

In [6]:
def returnDataProcess(move_data, startT, endT):
    """输入一周的单车移动数据，指定的日期与指定的时刻"""
    """输出该日期该时间切片内还车return位置"""

    data = move_data[(move_data['etime'] < endT) &
                     (move_data['etime'] >= startT)]
    data = data[['elon', 'elat']]

    data['geometry'] = geopandas.points_from_xy(data['elon'], data['elat'])

    data = geopandas.GeoDataFrame(data)
    data = data.set_crs('EPSG:4326')
    data = data.to_crs('EPSG:2416')

    data['x'] = data.geometry.x
    data['y'] = data.geometry.y

    data = data.to_crs('EPSG:4326')

    return data

## 单车停放热点区域

In [7]:
def stopDataProcess(stop_data, t):
    """输入一周的停车数据，指定的日期与指定的时刻"""
    """输出该日期该时刻处于停放状态的单车位置"""

    data = stop_data[(stop_data['stime'] <= t) & (stop_data['etime'] >= t)]
    data = data[['slon', 'slat']]

    data['geometry'] = geopandas.points_from_xy(data['slon'], data['slat'])

    data = geopandas.GeoDataFrame(data)
    data = data.set_crs('EPSG:4326')
    data = data.to_crs('EPSG:2416')

    data['x'] = data.geometry.x
    data['y'] = data.geometry.y

    data = data.to_crs('EPSG:4326')

    return data

# 小时级热点区域识别

## 热点区域识别函数

In [8]:
from sklearn.cluster import DBSCAN
import math
import warnings
warnings.filterwarnings("ignore")


def hotspotOutput(data, Time):

    pi = 3.141592653589793

    model = DBSCAN(eps=200, min_samples=130)

    model.fit(data[['x', 'y']].values)

    # 得到DBSCAN预测的分类便签
    labels = model.labels_

    temp = data.copy()
    temp['group'] = labels
    temp = temp[temp['group'] != -1]
    
    # 如果未聚类成功，全是离群点
    if temp.empty:
        print(str(Time)+"未聚类成功")
    else:
        temp = temp.dissolve("group").convex_hull.reset_index().set_geometry(0)\
            .rename(columns={0: "geometry"})

        # 计算面积
        temp['area'] = temp.to_crs(epsg=2416).area
        # 计算当量半径
        temp['radius'] = temp.apply(lambda r: round(
            math.sqrt(r['area']/pi), 2), axis=1)
        # 计算几何中心经纬度
        temp['x'] = temp['geometry'].centroid.x
        temp['y'] = temp['geometry'].centroid.y
        # 记录时间
        temp['time'] = str(Time)

    return temp

## 取车热点区域

In [ ]:
import numpy as np
import datetime

for day in np.arange(6,13):
    
    # 从06:00开始
    starttime = pd.to_datetime('2019-05-'+str(day)+' 06:00:00')
    
    # 清空结果dataframe
    datas = []
    
    # 每小时单车停放热点识别
    for hour in range(17):
        
        sTime = starttime + datetime.timedelta(hours=hour*1)
        eTime = sTime + datetime.timedelta(hours=1)
        # 提取数据
        data = pickupDataProcess(move_data, str(sTime), str(eTime))
        
        if data.empty:
            print(str(sTime)+"无数据")
        else:
            # 热点识别，输出热点区域相关形态位置指标
            res = hotspotOutput(data, sTime)
            # 添加至结果dataframe
            datas.append(res)

    datas = pd.concat(datas)
    datas = geopandas.GeoDataFrame(datas)
    
    datas.to_file(r'result\pickuphotspot\puh05'+str(day)+'.json', driver='GeoJSON')
    print("5月"+str(day)+"日热点结果已保存")

## 还车热点区域

In [ ]:
import numpy as np
import datetime

for day in np.arange(6,13):
    
    # 从06:00开始
    starttime = pd.to_datetime('2019-05-'+str(day)+' 06:00:00')
    
    # 清空结果dataframe
    datas = []
    
    # 每小时单车停放热点识别
    for hour in range(17):
        
        sTime = starttime + datetime.timedelta(hours=hour*1)
        eTime = sTime + datetime.timedelta(hours=1)
        # 提取数据
        data = returnDataProcess(move_data, str(sTime), str(eTime))
        
        if data.empty:
            print(str(sTime)+"无数据")
        else:
            # 热点识别，输出热点区域相关形态位置指标
            res = hotspotOutput(data, sTime)
            # 添加至结果dataframe
            datas.append(res)

    datas = pd.concat(datas)
    datas = geopandas.GeoDataFrame(datas)
    
    datas.to_file(r'result\returnhotspot\rh05'+str(day)+'.json', driver='GeoJSON')
    print("5月"+str(day)+"日热点结果已保存")

## 单车停放热点区域

In [ ]:
import numpy as np
import datetime

for day in np.arange(6,13):
    
    # 从06:00开始
    starttime = pd.to_datetime('2019-05-'+str(day)+' 06:00:00')
    
    # 清空结果dataframe
    datas = []
    
    # 每小时单车停放热点识别
    for hour in range(18):
        
        Time = starttime + datetime.timedelta(hours=hour*1)
        # 提取数据
        data = stopDataProcess(stop_data, str(Time))
        
        if data.empty:
            print(str(Time)+"无数据")
        else:
            # 热点识别，输出热点区域相关形态位置指标
            res = hotspotOutput(data, Time)
            # 添加至结果dataframe
            datas.append(res)

    datas = pd.concat(datas)
    datas = geopandas.GeoDataFrame(datas)
    
    datas.to_file(r'result\parkinghotspot\ph05'+str(day)+'.json', driver='GeoJSON')
    print("5月"+str(day)+"日热点结果已保存")